In [1]:
from IPython.display import display

from sympy import *

from monom import *

In [2]:
eps, t, h, a, phi, h = symbols(r'\varepsilon, t, h, a, \varphi, h', real=True)
u, u1 = (f(t) for f in symbols('u, u1', cls=Function))

In [3]:
eq = u.diff(t, 2)  + u + eps*u**3
eq

\varepsilon*u(t)**3 + u(t) + Derivative(u(t), (t, 2))

Первый интеграл 123

https://disk.yandex.ru/i/ng5ws4p2rYTcQA

In [4]:
eq1 = (u.diff(t)**2/2 + u**2/2 + eps*u**4/4) - h

In [5]:
expand(eq1.diff(t) - eq*u.diff(t))

0

Методика Линдштедта—Пуанкаре 129
Метод многих масштабов 137

In [6]:
s = t*(1 + 3*a**2*eps/8)
us = a*cos(s + phi) + eps/32*a**3*cos(3*(s + phi))

In [7]:
us.diff(eps, 0).subs(eps, 0).trigsimp()

a*cos(\varphi + t)

In [8]:
us.diff(eps, 1).subs(eps, 0).trigsimp()

a**3*(-12*t*sin(\varphi + t) + cos(3*(\varphi + t)))/32

In [9]:
us.diff(eps, 2).subs(eps, 0).trigsimp()

-9*a**5*t*(2*t*cos(\varphi + t) + sin(3*(\varphi + t)))/128

In [10]:
eq1 = eq.subs(u, us).doit()
eq1

\varepsilon*a**3*cos(3*\varphi + 3*t*(3*\varepsilon*a**2/8 + 1))/32 + \varepsilon*(\varepsilon*a**3*cos(3*\varphi + 3*t*(3*\varepsilon*a**2/8 + 1))/32 + a*cos(\varphi + t*(3*\varepsilon*a**2/8 + 1)))**3 - a*(3*\varepsilon*a**2 + 8)**2*(9*\varepsilon*a**2*cos(3*(\varphi + t*(3*\varepsilon*a**2 + 8)/8)) + 32*cos(\varphi + t*(3*\varepsilon*a**2 + 8)/8))/2048 + a*cos(\varphi + t*(3*\varepsilon*a**2/8 + 1))

In [11]:
eq1.diff(eps, 0).subs(eps, 0).trigsimp()

0

In [12]:
eq1.diff(eps, 1).subs(eps, 0).trigsimp()

0

In [13]:
eq1.diff(eps, 2).subs(eps, 0).trigsimp()

3*a**5*(-5*cos(\varphi + t) - 7*cos(3*(\varphi + t)) + cos(5*(\varphi + t)))/64

In [14]:
pda_f, pda_v = (u, u1), (t,)
pda_n, pda_clp = 9, 7
pda_p = Rational(0, 1),

In [15]:
Monom.variables = 1 + len(pda_v)
Monom.cmp = Monom.TOPdeglex
Monom.zero = Monom(0 for v in range(Monom.variables))
pda_fun = dict(zip(pda_f,\
             (Monom(0 if v else i for v in range(Monom.variables))\
              for i in range(1, 1 + len(pda_f)))))
pda_var = dict(zip(pda_v,\
             (Monom(0 if v != i else 1 for v in range(Monom.variables))\
              for i in range(1, Monom.variables))))

In [16]:
def T(f, j1):
    return sum(\
        diff(f, t, j)*(h*(j1-pda_p[0]))**j/\
                 (factorial(j))\
        for j in range(pda_n))

In [17]:
expand((T(u, 1)-T(u, -1))/(2*h))

h**6*Derivative(u(t), (t, 7))/5040 + h**4*Derivative(u(t), (t, 5))/120 + h**2*Derivative(u(t), (t, 3))/6 + Derivative(u(t), t)

In [18]:
def clip(f):
    f = f.expand()
    return [f.coeff(h, i) for i in range(pda_clp)]

def df2m(a):
    assert a.func == Derivative
    m = pda_fun[a.args[0]]
    for xi in a.args[1:]:
        if isinstance(xi, Symbol):
            m = m*pda_var[xi]
        else:
            m = m*pda_var[xi[0]]**xi[1]
    return m

def m2df(m):
    r = pda_f[m[0]-1]
    for i in range(1, len(m)):
        r = r.diff(pda_v[i-1], m[i])
    return r

def findDiv(a, d):
    r = None
    def find(a, r):
        if a.args:
            if a.func == Derivative and a.args[0] in pda_fun:
                m = df2m(a)
                if m.divisible(d) and (not r or m.cmp(r) > 0):
                    r = m
            else:
                for s in a.args:
                    r = find(s, r)
        return r
    return find(a, r)

def reduction(f1, f2, m, c, shift):
    assert shift < pda_clp
    r = [f1[i] for i in range(shift)]
    if not m:
        for i in range(shift, pda_clp):
            r.append(expand(f1[i] - f2[i-shift]*c))
    else:
        for i in range(shift, pda_clp):
            r.append(expand(f1[i] - f2[i-shift].diff(*m)*c))
    return r

def NF(f, df, G, head=False, trace=False):
    assert len(df) == len(G)
#     print(df2m(df[0]))
    ms = [df2m(d) for d in df]
    for i in range(0 if head else 1, pda_clp):
        t = 0
        if f[i]:
            while True:
                r = None
                for l in range(len(ms)):
                    r = findDiv(f[i], ms[l])
                    if r:
                        break
                if not r: 
                    break
                c, deg, m = 0, 7, m2df(r)
                while c == 0:
                    c = f[i].coeff(m, deg)
                    deg -= 1
                    assert deg >= 0
#                 print(c, m, deg+1)
                if deg:
                    c *= m**deg
                m = r/ms[l]
                d = []
                for k in range(len(pda_v)):
                    if m[k+1] > 0:
                        d.append(pda_v[k])
                        if m[k+1] > 1:
                            d.append(m[k+1])
                if trace:
                    print(">"*12)
                    eq = Symbol("eq%d" % (l+1), real=True)
                    if d:
                        display(Derivative(*tuple([eq] + d))*c*h**i)
                    else:
                        display(eq*c*h**i)
                f = reduction(f, G[l], tuple(d), c/G[l][0].coeff(df[l]), i)
                if trace:
                    print("res =")
                    display(f)
                    print("<"*12)
                t += 1
#                 if t > 6: break
    return f

def compact(f):
    def cmpct(a):
        if not a.args:
            return a
        else:
            if a in pda_fun:
                return Symbol("%s" % a.func, real=True)
            elif a.func == Derivative and a.args[0] in pda_fun:
                m = []
                for xi in a.args[1:]:
                    if isinstance(xi, Symbol):
                        m.append(str(xi))
                    else:
                        m.append(str(xi[0])*xi[1])
                return Symbol("%s_{%s}" % (a.args[0].func, "".join(m)), real=True)
            else:
                return a.func(*tuple(cmpct(s) for s in a.args))
    return cmpct(f)

def prn(a):
    for i in range(pda_clp):
        print("h^%d =>" % i)
        display(compact(a[i]).collect(eps, factor))
    
def prnlatex(a):
    print(latex(compact(a[0]).collect(eps, factor)))
    print(r"+h\big(")
    print(latex(compact(a[1]).collect(eps, factor))) 
    print(r"\big)")
    for i in range(2, pda_clp):
        print(r"+h^%d\big(" % i)
        print(latex(compact(a[i]).collect(eps, factor))) 
        print(r"\big)") 

In [19]:
def Dtt(a):
    return (T(a, 1) - 2*T(a, 0) + T(a, -1))/h**2
def Dt(a):
    return (T(a, 1) - T(a, -1))/h

In [20]:
s1 = expand(Dtt(u) + u + eps*T(u**3, 0))
s1

\varepsilon*u(t)**3 + h**6*Derivative(u(t), (t, 8))/20160 + h**4*Derivative(u(t), (t, 6))/360 + h**2*Derivative(u(t), (t, 4))/12 + u(t) + Derivative(u(t), (t, 2))

In [21]:
F1 = clip(s1)
prn(F1)

h^0 =>


\varepsilon*u**3 + u + u_{tt}

h^1 =>


0

h^2 =>


...

u_{tttttt}/360

h^5 =>


0

h^6 =>


u_{tttttttt}/20160

In [22]:
f1 = NF(F1, [u.diff(t, 2)], [F1], head=False)
prn(f1)
prnlatex(f1)

h^0 =>


\varepsilon*u**3 + u + u_{tt}

h^1 =>


0

h^2 =>


...

h^5 =>


0

h^6 =>


23*\varepsilon**4*u**9/120 + \varepsilon**3*u**5*(247*u**2 - 714*u_{t}**2)/560 + \varepsilon**2*u*(543*u**4 - 2060*u**2*u_{t}**2 + 420*u_{t}**4)/1680 + \varepsilon*u*(381*u**2 - 874*u_{t}**2)/5040 + u/560

\varepsilon u^{3} + u + u_{tt}
+h\big(
0
\big)
+h^2\big(
\frac{\varepsilon^{2} u^{5}}{4} + \frac{\varepsilon u \left(2 u^{2} - 3 u_{t}^{2}\right)}{6} + \frac{u}{12}
\big)
+h^3\big(
0
\big)
+h^4\big(
\frac{7 \varepsilon^{3} u^{7}}{40} + \frac{\varepsilon^{2} u^{3} \left(19 u^{2} - 39 u_{t}^{2}\right)}{60} + \frac{\varepsilon u \left(55 u^{2} - 114 u_{t}^{2}\right)}{360} + \frac{u}{90}
\big)
+h^5\big(
0
\big)
+h^6\big(
\frac{23 \varepsilon^{4} u^{9}}{120} + \frac{\varepsilon^{3} u^{5} \left(247 u^{2} - 714 u_{t}^{2}\right)}{560} + \frac{\varepsilon^{2} u \left(543 u^{4} - 2060 u^{2} u_{t}^{2} + 420 u_{t}^{4}\right)}{1680} + \frac{\varepsilon u \left(381 u^{2} - 874 u_{t}^{2}\right)}{5040} + \frac{u}{560}
\big)


In [23]:
pda2 = f1[2].subs(u, us).doit()

In [24]:
pda2.diff(eps, 0).subs(eps, 0).trigsimp()

a*cos(\varphi + t)/12

In [25]:
pda2.diff(eps, 1).subs(eps, 0).trigsimp()

a**3*(-4*t*sin(\varphi + t) + 16*cos(\varphi + t) + 27*cos(3*(\varphi + t)))/128

In [26]:
pda2.diff(eps, 2).subs(eps, 0).trigsimp()

a**5*(-768*t**2*cos(\varphi + t) - 384*t*sin(3*(\varphi + t)) + 256*(96*t*sin(\varphi + t) - 8*cos(3*(\varphi + t)))*sin(\varphi + t)**2 - 512*(96*t*sin(\varphi + t) - 8*cos(3*(\varphi + t)))*cos(\varphi + t)**2 - 12288*(4*t*cos(\varphi + t) + 4*sin(\varphi + t) + sin(3*(\varphi + t)))*sin(\varphi + t)*cos(\varphi + t) + 32768*cos(\varphi + t)**5)/65536

In [27]:
def RK4(f, y):
    k1 = f(y)
    k2 = f(y + h/2*k1)
    k3 = f(y + h/2*k2)
    k4 = f(y + h*k3)
    return expand((k1 + 2*k2 + 2*k3 + k4)/6)

In [28]:
def f(y):
    return Matrix([\
        y[1],\
        -y[0] - eps*y[0]**3,\
    ])
r = RK4(f, Matrix([T(u, 0), T(u1, 0)]))
r

Matrix([
[                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [29]:
F1 = clip((T(u, 1)-T(u, 0))/h - T(r[0], 0))
prn(F1)

h^0 =>


-u1 + u_{t}

h^1 =>


\varepsilon*u**3/2 + (u + u_{tt})/2

h^2 =>


...

-\varepsilon**2*u**4*u1/8 - \varepsilon*u1*(3*u**2 - u1**2)/24 + u_{ttttt}/120

h^5 =>


\varepsilon**3*u**7/32 + \varepsilon**2*u**3*(2*u**2 - u1**2)/32 + \varepsilon*u*(u - u1)*(u + u1)/32 + u_{tttttt}/720

h^6 =>


\varepsilon**3*u**6*u1/64 + \varepsilon**2*u**4*u1/32 + \varepsilon*u**2*u1/64 + u_{ttttttt}/5040

In [30]:
F2 = clip((T(u1, 1)-T(u1, 0))/h - T(r[1], 0))
prn(F2)

h^0 =>


\varepsilon*u**3 + u + u1_{t}

h^1 =>


3*\varepsilon*u**2*u1/2 + (u1 + u1_{tt})/2

h^2 =>


...

3*\varepsilon**3*u**7/16 + \varepsilon**2*u**3*(3*u**2 - 10*u1**2)/8 + \varepsilon*u*(3*u**2 - 10*u1**2)/16 + u1_{ttttt}/120

h^5 =>


17*\varepsilon**3*u**6*u1/32 + \varepsilon**2*u**2*u1*(26*u**2 - 25*u1**2)/32 + \varepsilon*u1*(27*u**2 - 13*u1**2)/96 + u1_{tttttt}/720

h^6 =>


-5*\varepsilon**4*u**9/192 - \varepsilon**3*u**5*(5*u**2 - 54*u1**2)/64 - \varepsilon**2*u*(5*u**4 - 56*u**2*u1**2 + 16*u1**4)/64 - 5*\varepsilon*u*(u**2 - 6*u1**2)/192 + u1_{ttttttt}/5040

In [31]:
f1 = NF(F1, [u.diff(t), u1.diff(t)], [F1, F2], head=False)
prn(f1)
prnlatex(f1)

h^0 =>


-u1 + u_{t}

h^1 =>


0

h^2 =>


...

h^5 =>


\varepsilon**3*u**7/16 + \varepsilon**2*u**3*(10*u**2 + 3*u1**2)/96 + \varepsilon*u*(14*u**2 + 3*u1**2)/288 + u/144

h^6 =>


5*\varepsilon**3*u**6*u1/192 + 5*\varepsilon**2*u**2*u1*(9*u**2 - 14*u1**2)/672 + \varepsilon*u1*(57*u**2 - 20*u1**2)/4032 - u1/336

- u_{1} + u_{t}
+h\big(
0
\big)
+h^2\big(
0
\big)
+h^3\big(
0
\big)
+h^4\big(
\frac{\varepsilon^{2} u^{4} u_{1}}{10} + \frac{\varepsilon u_{1} \left(3 u - u_{1}\right) \left(3 u + u_{1}\right)}{120} + \frac{u_{1}}{120}
\big)
+h^5\big(
\frac{\varepsilon^{3} u^{7}}{16} + \frac{\varepsilon^{2} u^{3} \left(10 u^{2} + 3 u_{1}^{2}\right)}{96} + \frac{\varepsilon u \left(14 u^{2} + 3 u_{1}^{2}\right)}{288} + \frac{u}{144}
\big)
+h^6\big(
\frac{5 \varepsilon^{3} u^{6} u_{1}}{192} + \frac{5 \varepsilon^{2} u^{2} u_{1} \left(9 u^{2} - 14 u_{1}^{2}\right)}{672} + \frac{\varepsilon u_{1} \left(57 u^{2} - 20 u_{1}^{2}\right)}{4032} - \frac{u_{1}}{336}
\big)


In [32]:
f2 = NF(F2, [u.diff(t), u1.diff(t)], [F1, F2], head=False)
prn(f2)
prnlatex(f2)

h^0 =>


\varepsilon*u**3 + u + u1_{t}

h^1 =>


0

h^2 =>


...

h^5 =>


-\varepsilon**2*u**2*u1*(2*u**2 - u1**2)/32 - \varepsilon*u1**3/96 + u1/144

h^6 =>


7*\varepsilon**4*u**9/64 + \varepsilon**3*u**5*(103*u**2 + 28*u1**2)/448 + \varepsilon**2*u*(213*u**4 - 12*u**2*u1**2 + 70*u1**4)/1344 + \varepsilon*u*(55*u**2 - 12*u1**2)/1344 + u/336

\varepsilon u^{3} + u + u1_{t}
+h\big(
0
\big)
+h^2\big(
0
\big)
+h^3\big(
0
\big)
+h^4\big(
- \frac{3 \varepsilon^{3} u^{7}}{80} - \frac{\varepsilon^{2} u^{3} \left(u^{2} + 4 u_{1}^{2}\right)}{20} - \frac{\varepsilon u \left(5 u^{2} + 18 u_{1}^{2}\right)}{240} - \frac{u}{120}
\big)
+h^5\big(
- \frac{\varepsilon^{2} u^{2} u_{1} \left(2 u^{2} - u_{1}^{2}\right)}{32} - \frac{\varepsilon u_{1}^{3}}{96} + \frac{u_{1}}{144}
\big)
+h^6\big(
\frac{7 \varepsilon^{4} u^{9}}{64} + \frac{\varepsilon^{3} u^{5} \left(103 u^{2} + 28 u_{1}^{2}\right)}{448} + \frac{\varepsilon^{2} u \left(213 u^{4} - 12 u^{2} u_{1}^{2} + 70 u_{1}^{4}\right)}{1344} + \frac{\varepsilon u \left(55 u^{2} - 12 u_{1}^{2}\right)}{1344} + \frac{u}{336}
\big)


In [33]:
F3 = clip((T(u1, 1)**2/2 + T(u, 1)**2/2 + eps*T(u, 1)**4/4) -\
          (T(u1, 0)**2/2 + T(u, 0)**2/2 + eps*T(u, 0)**4/4))
prn(F3)

h^0 =>


0

h^1 =>


\varepsilon*u**3*u_{t} + u*u_{t} + u1*u1_{t}

h^2 =>


...

\varepsilon*(u**3*u_{tttt} + 12*u**2*u_{ttt}*u_{t} + 9*u**2*u_{tt}**2 + 36*u*u_{tt}*u_{t}**2 + 6*u_{t}**4)/24 + (u*u_{tttt} + u1*u1_{tttt} + 4*u1_{ttt}*u1_{t} + 3*u1_{tt}**2 + 4*u_{ttt}*u_{t} + 3*u_{tt}**2)/24

h^5 =>


\varepsilon*(u**3*u_{ttttt} + 15*u**2*u_{tttt}*u_{t} + 30*u**2*u_{ttt}*u_{tt} + 60*u*u_{ttt}*u_{t}**2 + 90*u*u_{tt}**2*u_{t} + 60*u_{tt}*u_{t}**3)/120 + (u*u_{ttttt} + u1*u1_{ttttt} + 5*u1_{tttt}*u1_{t} + 10*u1_{ttt}*u1_{tt} + 5*u_{tttt}*u_{t} + 10*u_{ttt}*u_{tt})/120

h^6 =>


\varepsilon*(u**3*u_{tttttt} + 18*u**2*u_{ttttt}*u_{t} + 45*u**2*u_{tttt}*u_{tt} + 30*u**2*u_{ttt}**2 + 90*u*u_{tttt}*u_{t}**2 + 360*u*u_{ttt}*u_{tt}*u_{t} + 90*u*u_{tt}**3 + 120*u_{ttt}*u_{t}**3 + 270*u_{tt}**2*u_{t}**2)/720 + (u*u_{tttttt} + u1*u1_{tttttt} + 6*u1_{ttttt}*u1_{t} + 15*u1_{tttt}*u1_{tt} + 10*u1_{ttt}**2 + 6*u_{ttttt}*u_{t} + 15*u_{tttt}*u_{tt} + 10*u_{ttt}**2)/720

In [34]:
f3 = NF(F3, [u.diff(t), u1.diff(t)], [F1, F2], head=False)
prn(f3)
prnlatex(f3)

h^0 =>


0

h^1 =>


0

h^2 =>


...

h^5 =>


-\varepsilon**3*u**7*u1/16 - \varepsilon**2*u**3*u1*(3*u**2 - 5*u1**2)/24 - \varepsilon*u*u1*(3*u**2 - 4*u1**2)/48

h^6 =>


-\varepsilon**4*u**10/32 - \varepsilon**3*u**6*(7*u**2 + 54*u1**2)/96 - \varepsilon**2*u**2*(17*u**4 + 210*u**2*u1**2 - 81*u1**4)/288 - \varepsilon*(7*u**4 + 66*u**2*u1**2 - 15*u1**4)/288 - (u**2 + u1**2)/144

0
+h\big(
0
\big)
+h^2\big(
0
\big)
+h^3\big(
0
\big)
+h^4\big(
0
\big)
+h^5\big(
- \frac{\varepsilon^{3} u^{7} u_{1}}{16} - \frac{\varepsilon^{2} u^{3} u_{1} \left(3 u^{2} - 5 u_{1}^{2}\right)}{24} - \frac{\varepsilon u u_{1} \left(3 u^{2} - 4 u_{1}^{2}\right)}{48}
\big)
+h^6\big(
- \frac{\varepsilon^{4} u^{10}}{32} - \frac{\varepsilon^{3} u^{6} \left(7 u^{2} + 54 u_{1}^{2}\right)}{96} - \frac{\varepsilon^{2} u^{2} \left(17 u^{4} + 210 u^{2} u_{1}^{2} - 81 u_{1}^{4}\right)}{288} - \frac{\varepsilon \left(7 u^{4} + 66 u^{2} u_{1}^{2} - 15 u_{1}^{4}\right)}{288} - \frac{u^{2} + u_{1}^{2}}{144}
\big)
